## Annotation of OMERO data using napari-micro-sam

### Instructions:
  - To run with OMERO and to not expose login in the notebook, the username is stored in .env file (see example .env_example). The password needs to be typed everytime.
    Still it is not recommended to save credentials unencrypted, hence a better solution will be worked on.
  - This notebook supports processing images from various OMERO object types: images, datasets, projects, plates, and screens.
  - Specify the container type in the `datatype` variable and the object ID in the `data_id` variable.
  - You can choose to segment all images in the container or select a random subset for training and validation.
    - You can now specify multiple z-slices and timepoints to analyze.
    - You can extract and analyze patches from large images, to reduce the number of object to annotated and still annotate more images.

### TODOs
See [TODO.md](./TODO.md) for the complete list of planned improvements and features.

## Load all required packages and dependencies

In [54]:
# OMERO-related imports
import omero
from omero.gateway import BlitzGateway
import ezomero
from napari.settings import get_settings

# Scientific computing and image processing
import numpy as np
import pandas as pd

# File and system operations
import os
import shutil
import tempfile
import warnings
from dotenv import load_dotenv

import importlib
# Reload specific modules
import src.omero_functions
import src.file_io_functions
import src.image_functions
import src.utils
import src.processing_pipeline

importlib.reload(src.omero_functions)
importlib.reload(src.file_io_functions)
importlib.reload(src.image_functions)
importlib.reload(src.utils)
importlib.reload(src.processing_pipeline)

# Re-import after reloading
from src.omero_functions import print_object_details, get_images_from_container, get_dask_image, upload_rois_and_labels, initialize_tracking_table, update_tracking_table_rows, get_dask_image_multiple, get_dask_dimensions
from src.file_io_functions import zip_directory, store_annotations_in_zarr, zarr_to_tiff, cleanup_local_embeddings, organize_local_outputs, save_annotations_schema
from src.image_functions import label_to_rois, generate_patch_coordinates, extract_patch
from src.utils import NumpyEncoder, interleave_arrays
from src.processing_pipeline import process_omero_batch
import sys

# Add auto-reload capability for src module
import importlib.util

# Force reload of main src package if it's already loaded
if "src" in sys.modules:
    importlib.reload(sys.modules["src"])

# Define a helper function to reload modules more concisely
def reload_module(module_name):
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    return __import__(module_name)

# Reload all src submodules
src_modules = [
    "src.omero_functions",
    "src.file_io_functions",
    "src.image_functions", 
    "src.utils",
    "src.processing_pipeline"
]

for module in src_modules:
    reload_module(module)

# Re-import after reloading to ensure we have the latest versions

get_settings().application.ipy_interactive = False


output_directory = os.path.normcase(tempfile.mkdtemp())
print('Created temporary work directory: ', output_directory)

Created temporary work directory:  c:\users\maarten\appdata\local\temp\tmp6r90im4s


### Setup connection with OMERO

In [55]:
load_dotenv(override=True)
# Ask for password if not set
if not os.environ.get("PASSWORD"):
    from getpass import getpass
    os.environ["PASSWORD"] = getpass("Enter OMERO server password: ")

conn = BlitzGateway(host=os.environ.get("HOST"), username=os.environ.get("USER_NAME"), passwd=os.environ.get("PASSWORD"), group=os.environ.get("GROUP"), secure=True)

connection_status = conn.connect()
if connection_status:
    print("Connected to OMERO Server")
else:
    print("Connection to OMERO Server Failed")
conn.c.enableKeepAlive(60)

INFO:omero.gateway:created connection (uuid=c306629a-70ab-4ffe-ac6d-02a734e22f4e)
INFO:omero.util.Resources:Starting


Connected to OMERO Server


### Select your dataset and check its content

In [56]:
datatype = "project" # "screen", "plate", "project", "dataset", "image"
data_id = 101

# Validate that data_id matches datatype and print details
if datatype == "project":
    project = conn.getObject("Project", data_id)
    if project is None:
        raise ValueError(f"Project with ID {data_id} not found")
    print_object_details(conn, project, "project")
    
elif datatype == "plate":
    plate = conn.getObject("Plate", data_id)
    if plate is None:
        raise ValueError(f"Plate with ID {data_id} not found")
    print_object_details(conn, plate, "plate")
    
elif datatype == "dataset":
    dataset = conn.getObject("Dataset", data_id)
    if dataset is None:
        raise ValueError(f"Dataset with ID {data_id} not found")
    print_object_details(conn, dataset, "dataset")
    
elif datatype == "image":
    image = conn.getObject("Image", data_id)
    if image is None:
        raise ValueError(f"Image with ID {data_id} not found")
    print_object_details(conn, image, "image")

else:
    raise ValueError("Invalid datatype specified")

# Check for any training tables already present in the dataset


Project Details:
- Name: Senescence
- ID: 101
- Owner: root root
- Group: system
- Number of datasets: 3
- Total images: 6


### Choose a training data set name
Use a specific name if you want to resume from an existing table  
Or use the datetime format for a new training set

In [57]:
# Set a name for the training set
# Use a specific name if you want to resume from an existing table
# Or use the datetime format for a new training set
trainingset_name = "training_data_20240602f"  # Use a fixed name if resuming from an existing table
# trainingset_name = "training_data_" + pd.Timestamp.now().strftime("%Y%m%d_%H%M")
print('Training Set Name: ', trainingset_name)

Training Set Name:  training_data_20240602f


### Start batch annotation with optimized table management

In [58]:
# Get all images from the specified container
images, source_desc = get_images_from_container(conn, datatype, data_id)

# Configuration for batch processing
segment_all = False  # Process all images in the dataset?
train_n = 3  # Number of training images (if not segment_all)
validate_n = 3  # Number of validation images (if not segment_all)
model_type = 'vit_b_lm'  # SAM model to use
batch_size = 6  # Number of images to process at once
channel = 1  # Channel to segment (usually the nuclear/main stain channel), starting from 0
three_d = False  # Use 3D mode?
z_slices = [0]  # Which z-slices to process (if not three_d) list(range(11))
z_slice_mode = "random"  # "all", "random", or "specific"
timepoints = list(range(14))   # Which timepoints to process
timepoint_mode = "random"  # "all", "random", or "specific"
group_by_image = True  # Keep all z-slices and timepoints from the same image together
resume_from_table = True  # Set to True to resume from an existing tracking table

# Patch extraction settings
use_patches = True  # Extract and process patches instead of full images?
patch_size = (256, 256)  # Size of patches to extract
patches_per_image = 2  # Number of patches to extract per image
random_patches = True  # Extract random patches or centered patches?

# Optional read-only mode (for OMERO servers where you don't have write permissions)
read_only_mode = False  # Save annotations locally instead of uploading to OMERO
local_output_dir = "./omero_annotations"  # Directory to save local annotations

if read_only_mode:
    # Use trainingset_name in local output directory if provided
    if trainingset_name:
        local_output_dir = f"./omero_annotations_{trainingset_name}"
    os.makedirs(local_output_dir, exist_ok=True)
    
# Summarize the configuration for the user
print(f"Configuration Summary:")
print(f"  - Segment All Images: {segment_all}")
print(f"  - Training Images: {train_n}")
print(f"  - Validation Images: {validate_n}")
print(f"  - Model Type: {model_type}")
print(f"  - Batch Size: {batch_size}")
print(f"  - Channel: {channel}")
print(f"  - 3D Mode: {three_d}")
print(f"  - Z-Slices: {z_slices} (Mode: {z_slice_mode})")
print(f"  - Timepoints: {timepoints} (Mode: {timepoint_mode})")
print(f"  - Group Z/T by Image: {group_by_image}")
print(f"  - Use Patches: {use_patches}")
if(use_patches):
    print(f"  - Patch Size: {patch_size}")
    print(f"  - Patches per Image: {patches_per_image}")
    print(f"  - Random Patches: {random_patches}")
print(f"  - Read-Only Mode: {read_only_mode}")


Found 6 images from Project: Senescence (ID: 101)
Configuration Summary:
  - Segment All Images: False
  - Training Images: 3
  - Validation Images: 3
  - Model Type: vit_b_lm
  - Batch Size: 6
  - Channel: 1
  - 3D Mode: False
  - Z-Slices: [0] (Mode: random)
  - Timepoints: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] (Mode: random)
  - Group Z/T by Image: True
  - Use Patches: True
  - Patch Size: (256, 256)
  - Patches per Image: 2
  - Random Patches: True
  - Read-Only Mode: False


In [59]:
import ipywidgets as widgets
from IPython.display import display

# Get all images from the specified container
images, source_desc = get_images_from_container(conn, datatype, data_id)


# Create interactive widgets for configuration
def create_config_widget():
    # Basic processing settings
    segment_all_widget = widgets.Checkbox(value=False, description="Segment All Images")
    train_n_widget = widgets.IntSlider(
        value=3, min=1, max=20, description="Training Images"
    )
    validate_n_widget = widgets.IntSlider(
        value=3, min=1, max=20, description="Validation Images"
    )

    # Model and processing settings
    model_type_widget = widgets.Dropdown(
        options=["vit_b_lm", "vit_l_lm", "vit_h_lm"],
        value="vit_b_lm",
        description="Model Type",
    )
    batch_size_widget = widgets.IntSlider(
        value=6, min=1, max=20, description="Batch Size"
    )
    channel_widget = widgets.IntSlider(value=1, min=0, max=5, description="Channel")
    three_d_widget = widgets.Checkbox(value=False, description="3D Mode")

    # Z-slice settings
    z_slices_widget = widgets.Text(
        value="[0]", description="Z-Slices", placeholder="e.g., [0] or [0,1,2]"
    )
    z_slice_mode_widget = widgets.Dropdown(
        options=["all", "random", "specific"],
        value="random",
        description="Z-Slice Mode",
    )

    # Timepoint settings
    timepoints_widget = widgets.Text(
        value="list(range(14))",
        description="Timepoints",
        placeholder="e.g., [0,1,2] or list(range(14))",
    )
    timepoint_mode_widget = widgets.Dropdown(
        options=["all", "random", "specific"],
        value="random",
        description="Timepoint Mode",
    )

    resume_from_table_widget = widgets.Checkbox(
        value=True, description="Resume from Table"
    )
    
    # Group related slices setting
    group_by_image_widget = widgets.Checkbox(
        value=True, description="Group Z/T by Image",
        tooltip="Keep all slices and timepoints from the same image together in either training or validation sets"
    )

    # Patch extraction settings
    use_patches_widget = widgets.Checkbox(value=True, description="Use Patches")
    patch_width_widget = widgets.IntSlider(
        value=256, min=64, max=1024, step=64, description="Patch Width"
    )
    patch_height_widget = widgets.IntSlider(
        value=256, min=64, max=1024, step=64, description="Patch Height"
    )
    patches_per_image_widget = widgets.IntSlider(
        value=2, min=1, max=10, description="Patches per Image"
    )
    random_patches_widget = widgets.Checkbox(value=True, description="Random Patches")

    # Read-only mode settings
    read_only_mode_widget = widgets.Checkbox(value=False, description="Read-Only Mode")
    local_output_dir_widget = widgets.Text(
        value="./omero_annotations", description="Local Output Dir"
    )

    # Output area for configuration summary
    output = widgets.Output()

    def update_summary(*args):
        with output:
            output.clear_output()

            # Parse complex inputs
            try:
                z_slices = eval(z_slices_widget.value) if z_slices_widget.value else [0]
            except:
                z_slices = [0]

            try:
                timepoints = (
                    eval(timepoints_widget.value) if timepoints_widget.value else [0]
                )
            except:
                timepoints = [0]

            patch_size = (patch_width_widget.value, patch_height_widget.value)

            print("Configuration Summary:")
            print(f"  - Segment All Images: {segment_all_widget.value}")
            print(f"  - Training Images: {train_n_widget.value}")
            print(f"  - Validation Images: {validate_n_widget.value}")
            print(f"  - Model Type: {model_type_widget.value}")
            print(f"  - Batch Size: {batch_size_widget.value}")
            print(f"  - Channel: {channel_widget.value}")
            print(f"  - 3D Mode: {three_d_widget.value}")
            print(f"  - Z-Slices: {z_slices} (Mode: {z_slice_mode_widget.value})")
            print(f"  - Timepoints: {timepoints} (Mode: {timepoint_mode_widget.value})")
            print(f"  - Group Z/T by Image: {group_by_image_widget.value}")
            print(f"  - Use Patches: {use_patches_widget.value}")
            if use_patches_widget.value:
                print(f"  - Patch Size: {patch_size}")
                print(f"  - Patches per Image: {patches_per_image_widget.value}")
                print(f"  - Random Patches: {random_patches_widget.value}")
            print(f"  - Read-Only Mode: {read_only_mode_widget.value}")
            if read_only_mode_widget.value:
                print(f"  - Local Output Directory: {local_output_dir_widget.value}")

    # Observe changes in widgets
    widgets_to_observe = [
        segment_all_widget,
        train_n_widget,
        validate_n_widget,
        model_type_widget,
        batch_size_widget,
        channel_widget,
        three_d_widget,
        z_slices_widget,
        z_slice_mode_widget,
        timepoints_widget,
        timepoint_mode_widget,
        resume_from_table_widget,
        group_by_image_widget,
        use_patches_widget,
        patch_width_widget,
        patch_height_widget,
        patches_per_image_widget,
        random_patches_widget,
        read_only_mode_widget,
        local_output_dir_widget,
    ]

    for widget in widgets_to_observe:
        widget.observe(update_summary, names="value")

    # Initial summary
    update_summary()

    # Function to get current configuration
    def get_config():
        try:
            z_slices = eval(z_slices_widget.value) if z_slices_widget.value else [0]
        except:
            z_slices = [0]

        try:
            timepoints = (
                eval(timepoints_widget.value) if timepoints_widget.value else [0]
            )
        except:
            timepoints = [0]

        config = {
            "segment_all": segment_all_widget.value,
            "train_n": train_n_widget.value,
            "validate_n": validate_n_widget.value,
            "model_type": model_type_widget.value,
            "batch_size": batch_size_widget.value,
            "channel": channel_widget.value,
            "three_d": three_d_widget.value,
            "z_slices": z_slices,
            "z_slice_mode": z_slice_mode_widget.value,
            "timepoints": timepoints,
            "timepoint_mode": timepoint_mode_widget.value,
            "resume_from_table": resume_from_table_widget.value,
            "group_by_image": group_by_image_widget.value,
            "use_patches": use_patches_widget.value,
            "patch_size": (patch_width_widget.value, patch_height_widget.value),
            "patches_per_image": patches_per_image_widget.value,
            "random_patches": random_patches_widget.value,
            "read_only_mode": read_only_mode_widget.value,
            "local_output_dir": local_output_dir_widget.value,
        }
        return config

    # Layout the widgets
    basic_settings = widgets.VBox(
        [
            widgets.HTML("<h3>Basic Processing Settings</h3>"),
            segment_all_widget,
            train_n_widget,
            validate_n_widget,
            model_type_widget,
            batch_size_widget,
            channel_widget,
            three_d_widget,
        ]
    )

    slice_settings = widgets.VBox(
        [
            widgets.HTML("<h3>Slice & Timepoint Settings</h3>"),
            z_slices_widget,
            z_slice_mode_widget,
            timepoints_widget,
            timepoint_mode_widget,
            group_by_image_widget,
            resume_from_table_widget,
        ]
    )

    patch_settings = widgets.VBox(
        [
            widgets.HTML("<h3>Patch Extraction Settings</h3>"),
            use_patches_widget,
            patch_width_widget,
            patch_height_widget,
            patches_per_image_widget,
            random_patches_widget,
        ]
    )

    output_settings = widgets.VBox(
        [
            widgets.HTML("<h3>Output Settings</h3>"),
            read_only_mode_widget,
            local_output_dir_widget,
        ]
    )

    # Create tabs for better organization
    tab = widgets.Tab()
    tab.children = [basic_settings, slice_settings, patch_settings, output_settings]
    tab.titles = ["Basic", "Slices/Time", "Patches", "Output"]

    main_widget = widgets.VBox(
        [tab, widgets.HTML("<h3>Configuration Summary</h3>"), output]
    )

    return main_widget, get_config


# Create and display the widget
config_widget, get_config = create_config_widget()
display(config_widget)


Found 6 images from Project: Senescence (ID: 101)


In [60]:
# Get configuration from widget
config = get_config()

# Extract values for use in processing
segment_all = config["segment_all"]
train_n = config["train_n"]
validate_n = config["validate_n"]
model_type = config["model_type"]
batch_size = config["batch_size"]
channel = config["channel"]
three_d = config["three_d"]
z_slices = config["z_slices"]
z_slice_mode = config["z_slice_mode"]
timepoints = config["timepoints"]
timepoint_mode = config["timepoint_mode"]
group_by_image = config["group_by_image"]
resume_from_table = config["resume_from_table"]
use_patches = config["use_patches"]
patch_size = config["patch_size"]
patches_per_image = config["patches_per_image"]
random_patches = config["random_patches"]
read_only_mode = config["read_only_mode"]
local_output_dir = config["local_output_dir"]

# Handle read-only mode directory setup
if read_only_mode:
    if trainingset_name:
        local_output_dir = f"./omero_annotations_{trainingset_name}"
    os.makedirs(local_output_dir, exist_ok=True)

print("Configuration loaded from widget!")


Configuration loaded from widget!


### Run the annotation routine

In [ ]:
table_id, combined_images = process_omero_batch(
    conn,
    images,
    output_directory,
    datatype,
    data_id,
    source_desc,
    model_type=model_type,
    batch_size=batch_size,
    channel=channel,
    timepoints=timepoints,
    timepoint_mode=timepoint_mode,
    z_slices=z_slices,
    z_slice_mode=z_slice_mode,
    segment_all=segment_all,
    train_n=train_n,
    validate_n=validate_n,
    three_d=three_d,
    use_patches=use_patches,
    patch_size=patch_size,
    patches_per_image=patches_per_image,
    random_patches=random_patches,
    resume_from_table=resume_from_table,
    read_only_mode=read_only_mode,
    local_output_dir=local_output_dir,
    trainingset_name=trainingset_name,
    group_by_image=group_by_image
)

print(f"Annotation complete! Table ID: {table_id}")
print(f"Processed {len(combined_images)} images")

Found 1 file annotations on Project 101
No table found with title 'micro_sam_training_data_20240602f'
Table 'micro_sam_training_data_20240602f' not found, creating a new one
Interleaving training and validation images: 3 training images, 3 validation images
Creating complete tracking table with all planned images/patches...
Using table title: micro_sam_training_data_20240602f


c:\Users\Maarten\Documents\Github\omero_ipynb\microsam\src\omero_functions.py:685: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_for_omero[col] = df_for_omero[col].fillna(False).astype(bool)
c:\Users\Maarten\Documents\Github\omero_ipynb\microsam\src\omero_functions.py:685: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_for_omero[col] = df_for_omero[col].fillna(False).astype(bool)
c:\Users\Maarten\Documents\Github\omero_ipynb\microsam\src\omero_functions.py:685: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated a

Created tracking table with 12 rows, ID: 355
object group 0
Stored annotation configuration in OMERO with ID: 356


INFO:omero.gateway:Registered c306629a-70ab-4ffe-ac6d-02a734e22f4e/d669a121-cea2-409b-b117-2fbd2be36c0eomero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 41349 -t 60000


Processing 12 patches in 2 batches
3D mode: False

Processing batch 1/2


...

Loaded 2D image/patch for image 251 at timepoint 7, z-slice 0 with shape (256, 256)


INFO:omero.gateway:Unregistered c306629a-70ab-4ffe-ac6d-02a734e22f4e/eb874b39-fc13-405a-983a-ac6793fade5comero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 41349 -t 60000
INFO:omero.gateway:Registered c306629a-70ab-4ffe-ac6d-02a734e22f4e/b2ce615b-e0da-43ed-b783-e831f671ece4omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 41349 -t 60000


Loaded 2D image/patch for image 255 at timepoint 11, z-slice 0 with shape (256, 256)


INFO:omero.gateway:Unregistered c306629a-70ab-4ffe-ac6d-02a734e22f4e/b2ce615b-e0da-43ed-b783-e831f671ece4omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 41349 -t 60000


Loaded 2D image/patch for image 255 at timepoint 12, z-slice 0 with shape (256, 256)
Starting napari viewer with SAM annotator. Close the viewer window when done.


## Export training dataset

### Display annotations tracking table

In [ ]:
# Retrieve and display the tracking table
if table_id is not None:
    tracking_df = ezomero.get_table(conn, table_id)
    print(f"Tracking table contains {len(tracking_df)} rows")
    display(tracking_df)
else:
    print("No tracking table was created")

### Clean up and close connection

In [ ]:
# Clean up temporary directory
try:
    shutil.rmtree(output_directory)
    print(f"Removed temporary directory: {output_directory}")
except Exception as e:
    print(f"Error removing temporary directory: {e}")

# Close OMERO connection
conn.close()
print("OMERO connection closed")